In [1]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, f1_score,precision_score
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV

## Load Data

In [2]:
x_train = pd.read_csv('final_x_train.csv')
y_train = pd.read_csv('final_y_train.csv')
x_valid = pd.read_csv('final_x_valid.csv')
y_valid = pd.read_csv('final_y_valid.csv')
x_test = pd.read_csv('final_x_test.csv')
y_test = pd.read_csv('final_y_test.csv')

## Set up models

Applied all methods in both one-vs-one and one-vs-rest. Then, we will compare their performance with the original methods based on their accuracy rates

In [3]:
def Orign_ModeL(model):
    model.fit(x_train, y_train)
    y_pred = model.predict(x_valid)
    f1 = f1_score(y_valid, y_pred, average='micro')
    precision = precision_score(y_valid, y_pred, average='micro')
    recall = recall_score(y_valid, y_pred, average='micro')
    print(f"precision score: {precision:.4f}, recall score: {recall:.4f}, f1 score: {f1:.4f}")
    return model

In [4]:
def OVO_Model(model):
    OVOmodel = OneVsOneClassifier(model)
    OVOmodel.fit(x_train, y_train)
    y_pred = OVOmodel.predict(x_valid)
    f1 = f1_score(y_valid, y_pred, average='micro')
    precision = precision_score(y_valid, y_pred, average='micro')
    recall = recall_score(y_valid, y_pred, average='micro')
    print(f"precision score: {precision:.4f}, recall score: {recall:.4f}, f1 score: {f1:.4f}")
    return OVOmodel

In [5]:
def OVR_Model(model):
    OVRmodel = OneVsRestClassifier(model)
    OVRmodel.fit(x_train, y_train)
    y_pred = OVRmodel.predict(x_valid)
    f1 = f1_score(y_valid, y_pred, average='micro')
    precision = precision_score(y_valid, y_pred, average='micro')
    recall = recall_score(y_valid, y_pred, average='micro')
    print(f"precision score: {precision:.4f}, recall score: {recall:.4f}, f1 score: {f1:.4f}")
    return OVRmodel

In [3]:
# disable warnings
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

y_vaild1 = y_valid.copy()
y_train1 = y_train.copy()
y_test1 = y_test.copy()

## Model fitting

In [7]:
# Naive Bayes
NBOM =Orign_ModeL(GaussianNB())
NBOVO=OVO_Model(GaussianNB())
NBOVR=OVR_Model(GaussianNB())

precision score: 0.0114, recall score: 0.0114, f1 score: 0.0114
precision score: 0.0114, recall score: 0.0114, f1 score: 0.0114
precision score: 0.0143, recall score: 0.0143, f1 score: 0.0143


In [8]:
# KNN
KNNOM =Orign_ModeL(KNeighborsClassifier())
KNNOVO=OVO_Model(KNeighborsClassifier())
KNNOVR=OVR_Model(KNeighborsClassifier()) 

precision score: 0.1701, recall score: 0.1701, f1 score: 0.1701
precision score: 0.1929, recall score: 0.1929, f1 score: 0.1929
precision score: 0.1649, recall score: 0.1649, f1 score: 0.1649


In [10]:
# Random Forest
RFOM =Orign_ModeL(RandomForestClassifier())
RFOVO=OVO_Model(RandomForestClassifier())
RFOVR=OVR_Model(RandomForestClassifier()) 

C:\Users\a9958\AppData\Local\Temp\ipykernel_13240\3920633701.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(x_train, y_train)


precision score: 0.2231, recall score: 0.2231, f1 score: 0.2231
precision score: 0.2176, recall score: 0.2176, f1 score: 0.2176
precision score: 0.2154, recall score: 0.2154, f1 score: 0.2154


In [11]:
# XGBoost
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_valid = le.fit_transform(y_valid)
XGBOM =Orign_ModeL(XGBClassifier())
XGBOVO=OVO_Model(XGBClassifier())
XGBOVR=OVR_Model(XGBClassifier()) 

precision score: 0.2088, recall score: 0.2088, f1 score: 0.2088
precision score: 0.2149, recall score: 0.2149, f1 score: 0.2149
precision score: 0.2208, recall score: 0.2208, f1 score: 0.2208


## Model improving 

To improve accuracy, you can utilize the GridSearchCV package and loops to compare different hyperparameters and select the best configuration for better performance. Those approach will be employed for both Random Forest and XGBoost algorithms.

### Random Forest

In [20]:
#Setup a empty dict to save the best hyperparameters for future model improvemnt
parameters = {}

# First, trying to find the best hyperparameters in bootstrap, max depth, min samples split, 
# min samples split, max leaf nodes, and min samples leaf
param_grid = {
    "bootstrap": [True, False],
    "max_depth": range(5, 30, 5),
    "min_samples_split": [2, 5, 10],
    "max_leaf_nodes": range(5, 55, 10),
    "min_samples_leaf": range(1, 11, 2)
}

Hyper_param_search = GridSearchCV(estimator=RandomForestClassifier(random_state=0),
                       param_grid=param_grid, scoring='f1_micro', n_jobs=8, cv=5, verbose=2.5)
Hyper_param_search.fit(x_train, y_train1)

# Update the best hyperparameter to the parameters dict
parameters.update(Hyper_param_search.best_params_)
Hyper_param_search.best_estimator_, Hyper_param_search.best_params_, Hyper_param_search.best_score_

Fitting 5 folds for each of 750 candidates, totalling 3750 fits


C:\Users\a9958\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:926: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


(RandomForestClassifier(bootstrap=False, max_depth=10, max_leaf_nodes=25,
                        min_samples_split=10, random_state=0),
 {'bootstrap': False,
  'max_depth': 10,
  'max_leaf_nodes': 25,
  'min_samples_leaf': 1,
  'min_samples_split': 10},
 0.22136916186820219)

In [21]:
# Secondly, trying to find the best hyperparameters in n estimators, max features , and criterion.
# Also, apply the best hyperparameters from last result
param_grid = {
    "n_estimators": [100, 500, 1000],
    "max_features": ["sqrt", "log2", None],
    "criterion": ["gini", "entropy"]
}
Hyper_param_search = GridSearchCV(estimator=RandomForestClassifier(**parameters, random_state=0),
                       param_grid=param_grid, scoring='f1_micro', n_jobs=8, cv=5, verbose=2.5,error_score='raise')
Hyper_param_search.fit(x_train, y_train1)
parameters.update(Hyper_param_search.best_params_)
Hyper_param_search.best_estimator_, Hyper_param_search.best_params_, Hyper_param_search.best_score_

Fitting 5 folds for each of 18 candidates, totalling 90 fits


C:\Users\a9958\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:926: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


(RandomForestClassifier(bootstrap=False, max_depth=10, max_features='sqrt',
                        max_leaf_nodes=25, min_samples_split=10, random_state=0),
 {'criterion': 'gini', 'max_features': 'sqrt', 'n_estimators': 100},
 0.22136916186820219)

In [27]:
# employed the best hyperparameters from previous result to baseline model, 
# One Vs One model and One Vs Rest model
Classifier = RandomForestClassifier(**parameters,random_state=0)
OvO = OneVsOneClassifier(RandomForestClassifier(**parameters,random_state=0))
OVR =OneVsRestClassifier(RandomForestClassifier(**parameters,random_state=0)) 

In [28]:
# Model evaluation by test set

#Baseline 
Classifier.fit(x_train,y_train)
y_pred = Classifier.predict(x_test)
f1 = f1_score(y_test1, y_pred, average='micro')
precision = precision_score(y_test, y_pred, average='micro')
recall = recall_score(y_test1, y_pred, average='micro')
print(f"For orignal method, precision score: {precision:.4f}, recall score: {recall:.4f}, f1 score: {f1:.4f}")

# One-Vs-One
OvO.fit(x_train,y_train)
y_pred = OvO.predict(x_test)
f1 = f1_score(y_test1, y_pred, average='micro')
precision = precision_score(y_test, y_pred, average='micro')
recall = recall_score(y_test1, y_pred, average='micro')
print(f"For Ove Vs One, precision score: {precision:.4f}, recall score: {recall:.4f}, f1 score: {f1:.4f}")

# One-Vs-Rest
OVR.fit(x_train,y_train)
y_pred = OVR.predict(x_test)
f1 = f1_score(y_test1, y_pred, average='micro')
precision = precision_score(y_test1, y_pred, average='micro')
recall = recall_score(y_test1, y_pred, average='micro')
print(f"For Ove vs Rest, precision score: {precision:.4f}, recall score: {recall:.4f}, f1 score: {f1:.4f}")

C:\Users\a9958\AppData\Local\Temp\ipykernel_13240\2617743496.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  Classifier.fit(x_train,y_train)


For Ove Vs One, precision score: 0.2354, recall score: 0.2354, f1 score: 0.2354
For Ove Vs One, precision score: 0.2201, recall score: 0.2201, f1 score: 0.2201
For Ove vs Rest, precision score: 0.2236, recall score: 0.2236, f1 score: 0.2236


##### Conclusion

In conclusion, after implementing various improvements, it was observed that the baseline method achieved the highest F1 score of 0.2354 for the Random Forest algorithm.

### XGBoost

Utilizing a loop to discover optimal hyperparameters for max depth, gamma, and reg alpha, while concurrently maintaining a dataframe to log each selected hyperparameter and its corresponding accuracy.

In [18]:
#Used to search the best result
def best_result(df):
    max_index = df['f1_score'].idxmax()
    max_row = df.loc[max_index]
    return max_row

In [7]:
max_depth = [3,4,5]
gamma = [0,0.1,0.2,0.3,0.4,0.5]
reg_alpha = [0.0001, 0.001, 0.01, 0.1, 1, 10]
conclusion = pd.DataFrame(columns=['max_depth', 'gamma', 'reg_alpha','f1_score'])

# Baseline 
for i in max_depth:
    for j in gamma:
        for k in reg_alpha:
            model = XGBClassifier(max_depth = i, gamma = j, reg_alpha = k,learning_rate=0.1, nthread=8, random_state=0)
            model.fit(x_train, y_train)
            y_pred = model.predict(x_valid)
            f1 = f1_score(y_valid, y_pred, average='micro')
            data = {'max_depth':i, 'gamma':j, 'reg_alpha':k,'f1_score': round(f1*100,4)}
            conclusion = pd.concat([conclusion, pd.DataFrame([data])], ignore_index=True)

In [19]:
best_result(conclusion)

max_depth          4
gamma            0.2
reg_alpha        1.0
f1_score     22.6291
Name: 52, dtype: object

In [9]:
#Using one-vs-one method 
conclusion1 = pd.DataFrame(columns=['max_depth', 'gamma', 'reg_alpha','f1_score'])

for i in max_depth:
    for j in gamma:
        for k in reg_alpha:
            model = OneVsOneClassifier(XGBClassifier(max_depth = i, gamma = j, reg_alpha = k,learning_rate=0.1, nthread=8, random_state=0))
            model.fit(x_train, y_train)
            y_pred = model.predict(x_valid)
            f1 = f1_score(y_valid, y_pred, average='micro')
            data = {'max_depth':i, 'gamma':j, 'reg_alpha':k,'f1_score': round(f1*100,4)}
            conclusion1 = pd.concat([conclusion, pd.DataFrame([data])], ignore_index=True)

In [20]:
best_result(conclusion1)

max_depth          4
gamma            0.2
reg_alpha        1.0
f1_score     22.6291
Name: 52, dtype: object

In [11]:
#Using one-vs-rest method 
conclusion2 = pd.DataFrame(columns=['max_depth', 'gamma', 'reg_alpha','f1_score'])

for i in max_depth:
    for j in gamma:
        for k in reg_alpha:
            model = OneVsRestClassifier(XGBClassifier(max_depth = i, gamma = j, reg_alpha = k,learning_rate=0.1, nthread=8, random_state=0))
            model.fit(x_train, y_train)
            y_pred = model.predict(x_valid)
            f1 = f1_score(y_valid, y_pred, average='micro')
            data = {'max_depth':i, 'gamma':j, 'reg_alpha':k,'f1_score': round(f1*100,4)}
            conclusion2 = pd.concat([conclusion, pd.DataFrame([data])], ignore_index=True)

In [21]:
best_result(conclusion2)

max_depth          5
gamma            0.5
reg_alpha       10.0
f1_score     22.9702
Name: 108, dtype: object

In [24]:
# Model evaluation by test set
y_test = le.fit_transform(y_test)

#Baseline 
model = XGBClassifier(max_depth = 4, gamma = 0.2, reg_alpha = 1 ,learning_rate=0.1, nthread=8, random_state=0)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
f1 = f1_score(y_test, y_pred, average='micro')
precision = precision_score(y_test, y_pred, average='micro')
recall = recall_score(y_test, y_pred, average='micro')
print(f"precision score: {precision:.4f}, recall score: {recall:.4f}, f1 score: {f1:.4f}")

#One-VS-One
OVOmodel = OneVsOneClassifier(XGBClassifier(max_depth = 4, gamma = 0.2, reg_alpha = 1 ,learning_rate=0.1, nthread=8, random_state=0))
OVOmodel.fit(x_train, y_train)
y_pred = OVOmodel.predict(x_test)
f1 = f1_score(y_test, y_pred, average='micro')
precision = precision_score(y_test, y_pred, average='micro')
recall = recall_score(y_test, y_pred, average='micro')
print(f"precision score: {precision:.4f}, recall score: {recall:.4f}, f1 score: {f1:.4f}")

#One-Vs-Rest
OVRmodel = OneVsRestClassifier(XGBClassifier(max_depth = 5, gamma = 0.5, reg_alpha = 10 ,learning_rate=0.1, nthread=8, random_state=0))
OVRmodel.fit(x_train, y_train)
y_pred = OVRmodel.predict(x_test)
f1 = f1_score(y_test, y_pred, average='micro')
precision = precision_score(y_test, y_pred, average='micro')
recall = recall_score(y_test, y_pred, average='micro')
print(f"precision score: {precision:.4f}, recall score: {recall:.4f}, f1 score: {f1:.4f}")

precision score: 0.2165, recall score: 0.2165, f1 score: 0.2165
precision score: 0.2078, recall score: 0.2078, f1 score: 0.2078
precision score: 0.2272, recall score: 0.2272, f1 score: 0.2272


##### Conclusion

In conclusion, after implementing various improvements, it was observed that the one-vs-rest method achieved the highest F1 score of 0.2272 for the XGBoost algorithm.